# KW_MMDS - Colab 1
## Wordcount in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

1. Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
# YOUR CODE HERE, USE THIS CELL ONLY (DO NOT USE ADDITIONAL CELLS.)
RDDs = sc.textFile('pg100.txt').map(lambda line: line.lower())\

                              .flatMap(lambda line: line.split(" "))\

                              .filter(lambda word: len(word) > 0)\

                              .filter(lambda word: word[0].isalpha())\

                              .map(lambda word: (word[0], 1))\

                              .reduceByKey(lambda a, b: a + b)\

                              .sortByKey()                             



RDDs.take(26)

2. Now, you have to work on the *1342-0.txt* file which contains 'the Pride and Prejudice' by Jane Austen. Write a Spark application which outputs the 10 most frequently used words that start with **uppercase**. 

In [ ]:
# 오만과 편견 다운로드: 1342-0.txt
pride_id='11HHiWgRiXHuNehW8AXrHarKJfGX1HHnB'
pride_downloaded = drive.CreateFile({'id': pride_id})
pride_downloaded.GetContentFile('1342-0.txt')

In [ ]:
# YOUR CODE HERE, USE THIS CELL ONLY (DO NOT USE ADDITIONAL CELLS.)
RDDs = sc.textFile('1342-0.txt').flatMap(lambda line: line.split(" "))\
                              .filter(lambda word: len(word) > 0)\
                              .filter(lambda word: word[0].isupper())\
                              .map(lambda word: (word, 1))\
                              .reduceByKey(lambda a, b: a + b)\
                              .map(lambda tuple: (tuple[1], tuple[0]))\
                              .sortByKey(ascending=False)
RDDs.take(10)